In [2]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms
import torch.nn.functional as F
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test, ON_THRESHOLD
from sklearn.metrics import mean_absolute_error
import os
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path

In [ ]:
gid = 0
srv = 1
c = 0
cmd = {}
cnn_tree_valid_pred = {}
num_iterations = 20000
lr = 0.01
p = 0
dataset = 5
for fold_num in range(5):
    cnn_tree_valid_pred[fold_num] = {}
    for lr in [0.01]:
        cnn_tree_valid_pred[fold_num][lr] = {}
        for p in [0, 0.2, 0.4, 0.8]:
            cnn_tree_valid_pred[fold_num][lr][p] = {}
            for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw', 'residual'])):

#                 if order[0] == 'hvac':
#                     continue

                cnn_tree_valid_pred[fold_num][lr][p][order] = {}


                o = "\', \'".join(str(x) for x in order)
                directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/{}/".format(dataset, fold_num, lr, p)
                filename = "valid-pred-[\'{}\'].npy".format(o)

                full_path = directory + filename
                my_file = Path(full_path)
                if not my_file.exists():
                    o = " ".join(str(x) for x in order)
                    line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 3 {} {} 0 {} {} &".format(gid, lr, num_iterations, fold_num, o))
#                     print(line)

                    c += 1
                    if c%1 == 0:
                        gid+=1
                    if gid == 4:
                        gid = 0
                        srv += 1
                else:
                    k = np.load(full_path).item()
                    for it in range(1000, 20001, 1000):
                        cnn_tree_valid_pred[fold_num][lr][p][order][it] = {}
                        for idx, appliance in enumerate(order):
                            cnn_tree_valid_pred[fold_num][lr][p][order][it][appliance] = k[it][idx]

print(c)                               

In [9]:
cnn_tree_valid_pred[0][0.01]

{0: {},
 ('fridge', 'hvac', 'dr', 'dw', 'mw'): {},
 ('fridge', 'hvac', 'dr', 'mw', 'dw'): {},
 ('fridge', 'hvac', 'dw', 'dr', 'mw'): {},
 ('fridge', 'hvac', 'dw', 'mw', 'dr'): {},
 ('fridge', 'hvac', 'mw', 'dr', 'dw'): {},
 ('fridge', 'hvac', 'mw', 'dw', 'dr'): {},
 ('fridge', 'dr', 'hvac', 'dw', 'mw'): {},
 ('fridge', 'dr', 'hvac', 'mw', 'dw'): {},
 ('fridge', 'dr', 'dw', 'hvac', 'mw'): {},
 ('fridge', 'dr', 'dw', 'mw', 'hvac'): {},
 ('fridge', 'dr', 'mw', 'hvac', 'dw'): {},
 ('fridge', 'dr', 'mw', 'dw', 'hvac'): {},
 ('fridge', 'dw', 'hvac', 'dr', 'mw'): {},
 ('fridge', 'dw', 'hvac', 'mw', 'dr'): {},
 ('fridge', 'dw', 'dr', 'hvac', 'mw'): {},
 ('fridge', 'dw', 'dr', 'mw', 'hvac'): {},
 ('fridge', 'dw', 'mw', 'hvac', 'dr'): {},
 ('fridge', 'dw', 'mw', 'dr', 'hvac'): {},
 ('fridge', 'mw', 'hvac', 'dr', 'dw'): {},
 ('fridge', 'mw', 'hvac', 'dw', 'dr'): {},
 ('fridge', 'mw', 'dr', 'hvac', 'dw'): {},
 ('fridge', 'mw', 'dr', 'dw', 'hvac'): {},
 ('fridge', 'mw', 'dw', 'hvac', 'dr'): {},
 ('